In [1]:
import pandas as pd 
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [17]:
    df = pd.read_csv("sphist.csv")
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values(by="Date", ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [15]:
df['date'] = pd.to_datetime(df['Date'])

In [4]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,date
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068,2015-12-07
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2015-12-04
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2015-12-03
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2015-12-02
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2015-12-01


In [5]:
df.sort_values(by='Date', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,date
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,1950-01-03
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,1950-01-04
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,1950-01-05
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,1950-01-06
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,1950-01-09


Datasets taken from the stock market need to be handled differently than datasets from other sectors when it's time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential and each observation comes a day after the previous observation. Thus, the observations are not all independent and you can't treat them as such.

This means you have to be extra careful not to inject "future" knowledge into past rows when you train and predict. Injecting future knowledge makes our model look good when we train and test it, but it fails in the real world. This is how many algorithmic traders lose money.

The time series nature of the data means that we can generate indicators to make our model more accurate. For instance, you can create a new column that contains the average price of the last 10 trades for each row. This incorporates information from multiple prior rows into one and makes predictions much more accurate.

When you do this, you have to be careful not to use the current row in the values you average. You want to teach the model how to predict the current price from historical prices. If you include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where you don't know the price upfront.

Here are some indicators that are interesting to generate for each row:

The average price from the past 5 days.
The average price for the past 30 days.
The average price for the past 365 days.
The ratio between the average price for the past 5 days, and the average price for the past 365 days.
The standard deviation of the price over the past 5 days.
The standard deviation of the price over the past 365 days.
The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.
"Days" means "trading days" -- so if you're computing the average of the past 5 days, it should be the 5 most recent dates before the current one. Assume that "price" means the Close column. Always be careful not to include the current price in these indicators! You're predicting the next day price, so our indicators are designed to predict the current price from the previous prices.

Some of these indicators require a year of historical data to compute. Our first day of data falls on 1950-01-03, so the first day you can start computing indicators on is 1951-01-03.

To compute indicators, you'll need to loop through each day from 1951-01-03 to 2015-12-07 (the last day you have prices for). For instance, if we were computing the average price from the past 5 days, we'd start at 1951-01-03, get the prices for each day from 1950-12-26 to 1951-01-02, and find the average. The reason why we start on the 26th and take more than 5 calendar days into account is because the stock market is shutdown on certain holidays. Since we're looking at the past 5 trading days, we need to look at more than 5 calendar days to find them. 

## creating indicators

In [18]:
df['avg_5'] = df['Close'].rolling(5).mean().shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN


In [19]:
df['avg_30'] = df['Close'].rolling(30).mean().shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN


In [20]:
df['avg_365'] = df['Close'].rolling(365).mean().shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN


In [21]:
df['std_5'] = df['Close'].rolling(5).std().shift(1)
df['std_365'] = df['Close'].rolling(365).std().shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN


In [22]:
df['avg_5/avg_365'] = df['avg_5']/df['avg_365']
df['std_5/std_365'] = df['std_5']/df['std_365']
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,avg_5/avg_365,std_5/std_365
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.tail()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,avg_5/avg_365,std_5/std_365
16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178,3.916109,64.370261,1.025297,0.060837
16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082,7.956808,64.352527,1.026680,0.123644
16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356,9.333599,64.277554,1.025573,0.145208
16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343,19.599946,64.121622,1.021580,0.305668
16589,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068,2080.771973,2080.237329,2036.869425,19.806136,64.058862,1.021554,0.309187


Since you're computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. Some of the indicators use 365 days of historical data and the dataset starts on 1950-01-03. Thus, any rows that fall before 1951-01-03 don't have enough historical data to compute all the indicators

In [24]:
df.dropna(axis=0, inplace=True)

In [25]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,avg_5/avg_365,std_5/std_365
365,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,21.703333,19.447726,0.256223,1.790253,1.120954,0.143121
366,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,21.683000,19.462411,0.213659,1.789307,1.125246,0.119409
367,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,21.659667,19.476274,0.092574,1.788613,1.128142,0.051758
368,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,21.631000,19.489562,0.115108,1.787659,1.126757,0.064390
369,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,21.599000,19.502082,0.204132,1.786038,1.121008,0.114293


## creating train and test dataframes 

In [26]:
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

## making predictions with linear regression

In [27]:
lr = LinearRegression()
lr.fit(train[["avg_5", "avg_30", "avg_365", "std_5", "std_365", "avg_5/avg_365", "std_5/std_365"]], train["Close"])
predictions = lr.predict(test[["avg_5", "avg_30", "avg_365", "std_5", "std_365", "avg_5/avg_365", "std_5/std_365"]])

## # Calculate error metrics

In [29]:
mae = mean_absolute_error(test["Close"], predictions)
mse = mean_squared_error(test["Close"], predictions)
print("MAE: ", mae)
print("MSE: ", mse)


       

MAE:  16.145140609743393
MSE:  492.9230344450363


In [30]:
print(df.head(15))


          Date       Open       High        Low      Close     Volume  \
365 1951-06-19  22.020000  22.020000  22.020000  22.020000  1100000.0   
366 1951-06-20  21.910000  21.910000  21.910000  21.910000  1120000.0   
367 1951-06-21  21.780001  21.780001  21.780001  21.780001  1100000.0   
368 1951-06-22  21.549999  21.549999  21.549999  21.549999  1340000.0   
369 1951-06-25  21.290001  21.290001  21.290001  21.290001  2440000.0   
370 1951-06-26  21.299999  21.299999  21.299999  21.299999  1260000.0   
371 1951-06-27  21.370001  21.370001  21.370001  21.370001  1360000.0   
372 1951-06-28  21.100000  21.100000  21.100000  21.100000  1940000.0   
373 1951-06-29  20.959999  20.959999  20.959999  20.959999  1730000.0   
374 1951-07-02  21.100000  21.100000  21.100000  21.100000  1350000.0   
375 1951-07-03  21.230000  21.230000  21.230000  21.230000  1250000.0   
376 1951-07-05  21.639999  21.639999  21.639999  21.639999  1410000.0   
377 1951-07-06  21.639999  21.639999  21.639999  21

In [31]:
print(df[df["Date"] == datetime(year=1951, month=1, day=2)].index)

Int64Index([], dtype='int64')
